# Project Analysis Prototype Notebook

This notebook is a testbed for approaches to the project analysis workflow.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

from pysal.explore import esda
from pysal.lib import weights

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [ ]:
import seaborn as sns
sns.set(style='white',font_scale=1.0,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))

In [ ]:
max(0, np.log(1))

In [ ]:
from mapswipe.workflows.project_remap import _get_user_metrics, _get_project_agg_weighted
from mapswipe.data_access import get_project_data  # todo replace with live call + augmentation

df_user_metrics = _get_user_metrics()

This project has a good mix of attributes:
* Many buildings grouped in varying densities
* Large and small buildings

In [ ]:
project_id = "-NEaR6DbJAbkpYJ_BDCH"
proj_data = get_project_data(project_id)
df_full = proj_data["full"]
df_agg = proj_data["agg"]
df_agg["project_id"] = project_id

In [ ]:
df_full.head()

In [ ]:
df_user_metrics.head()

In [ ]:
df_agg.head()

In [ ]:
df_agg_w = _get_project_agg_weighted(df_agg, df_full, df_user_metrics)

In [ ]:
len(df_agg_w[(df_agg_w["0_share_uw"] > df_agg_w["1_share_uw"]) & (df_agg_w["1_share"] > df_agg_w["0_share"])])

In [ ]:
df_agg_w.head()

In [ ]:
len(df_agg_w.at[0, "geometry"].geoms[0].interior.coords) - 1

In [ ]:
len(df_agg_w.at[0, "geometry"].geoms[0].interiors)

In [ ]:
dir(df_agg_w.at[0, "geometry"].geoms[0])

In [ ]:
df_agg_w["geometry"].bounds

In [ ]:
df_agg_w["remap_score"].describe()

In [ ]:
df_agg_w["coverage_ratio"].describe()

In [ ]:
len(df_agg_w[df_agg_w["building_area_m2"] > 1000.0])

# Agreement Analysis

In [ ]:
agreement_example = "t17004"

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

df_plot = df_agg_w
#df_plot = df_agg_w[df_agg_w["building_area_m2"] < 200.0]

plt.figure(figsize=(10,7))
plt.scatter(
    x=df_plot["agreement"],
    y=df_plot["remap_score"],
    s=4,
    #c=df_plot["building_area_m2"].apply(np.log),
    #c=df_plot["building_area_m2"],
    #cmap="coolwarm"
)

agreement, remap_score = df_agg_w[df_agg_w["task_id"] == agreement_example][["agreement", "remap_score"]].iloc[0, [0, 1]]
plt.scatter(x=agreement, y=remap_score, color="red", s=30)

plt.title("Agreement can be misleading", fontsize=20)
plt.xlabel("Agreement Score")
plt.ylabel("Remap Score (higher means more likely to need remapping")
plt.show()

# model = ols('remap_score ~ agreement', data=df_plot).fit()
# plt.plot(df_plot["agreement"], model.predict(df_plot["agreement"]), color='red')

In [ ]:
df_plot[(df_plot["agreement"] > 0.5) & (df_plot["remap_score"] > 0.7) & (df_plot["2_count"] > 1)].sort_values(["remap_score", "agreement"]).head()

In [ ]:
df_agg_w[df_agg_w["task_id"] == agreement_example].T

# Moran

In [ ]:
def moran_sig_quads(ser_tasks, lisa):
    sig = 1 * (lisa.p_sim < 0.05)
    spots = lisa.q * sig
    return pd.Series(spots, index=ser_tasks)

def calc_moran_local_for_dist(gdf_agg, col_name, dist_vals):
    moran_vals = {}
    # Project to UTM for distance calculation
    task_ids = gdf_agg["task_id"]
    gdf = gdf_agg.to_crs(gdf_agg.estimate_utm_crs())
    for dist in dist_vals:
        w = weights.DistanceBand.from_dataframe(gdf, threshold=dist)
        w.transform = "R"
        moran = esda.moran.Moran_Local(gdf[col_name], w)
        moran_vals[f"moran_quad_{int(dist)}m"] = moran_sig_quads(task_ids, moran)
    return pd.DataFrame(data=moran_vals, index=task_ids)

In [ ]:
df_moran_local = calc_moran_local_for_dist(df_agg, "incorrect_score", [500.0])

In [ ]:
for c in [c for c in df_moran_local.columns if c.startswith("moran_quad_")]:
    print("\n", df_moran_local[c].value_counts())

In [ ]:
df_moran_local.head(20)

In [ ]:
#df_moran_local_w = calc_moran_local_for_dist(df_agg_w, "remap_score", [150.0, 350.0])
df_moran_local_w = calc_moran_local_for_dist(df_agg_w, "remap_score", [500.0])

In [ ]:
for c in [c for c in df_moran_local_w.columns if c.startswith("moran_quad_")]:
    print("\n", df_moran_local_w[c].value_counts())

# Viz

In [ ]:
import folium
from scipy import stats
import h3
from shapely.geometry import Polygon
import geopandas as gpd
from folium.features import GeoJsonTooltip
from typing import Iterable
import branca.colormap as cm
from branca.element import MacroElement
from jinja2 import Template

# LISA colors
lc = {
    "ns": "#5c5c5c", # Values of 0
    "HH": "#d7191c",  # Values of 1
    "LH": "#abd9e9",  # Values of 2
    "LL": "#2c7bb6",  # Values of 3
    "HL": "#fdae61",  # Values of 4
}
lisa_colormap = [lc["ns"], lc["HH"], lc["LH"], lc["LL"], lc["HL"]]


class Legend(MacroElement):
    def __init__(self, color_map, title="Legend"):
        super().__init__()
        self._template = Template('''
        {% macro html(this, kwargs) %}
        <div style="
            position: fixed; 
            bottom: 50px; 
            right: 50px; 
            width: 120px; 
            height: auto; 
            background-color: white;
            border: 2px solid grey; 
            z-index:9999; 
            font-size:14px;
            ">
            <p style="text-align: center;"><strong>{{this.title}}</strong></p>
            {% for category, color in this.color_map.items() %}
            <p>
                <i class="fa fa-square" style="color:{{color}}"></i> {{category}}
            </p>
            {% endfor %}
        </div>
        {% endmacro %}
        ''')
        self.color_map = color_map
        self.title = title



def create_moran_quad_map(gdf_agg, color_col, value_cols, center_pt=None):
    gdf = gdf_agg.copy()
    #gdf[color_col] = gdf[color_col].astype(str)

    geojson_data = gdf.drop('lastEdit', axis=1).to_json()

    if center_pt is None:
        center_pt = gdf.to_crs(gdf.estimate_utm_crs()).dissolve().centroid.to_crs(4326)
    map = folium.Map(tiles="Esri.WorldImagery", location=[center_pt.y, center_pt.x], zoom_start=8)
    map._repr_html_ = lambda: map._parent._repr_html_(
    include_link=False, width='75%', height='400px'
    )

    tooltip = GeoJsonTooltip(
        fields=[color_col] + [v for v in value_cols],
        aliases=[color_col] + [f"{v} Value" for v in value_cols],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    
    def style_function(feature):
        fillval = feature['properties'][color_col]
        fillval = int(fillval)
        return {
            'fillColor': lisa_colormap[fillval],
            'color': 'black',
            'weight': 0.25,
            'fillOpacity': 0.8
        }

    folium.GeoJson(
        geojson_data,
        style_function=style_function,
        tooltip=tooltip,
        name="geojson"
    ).add_to(map)

    #colormap.add_to(map)

    return map


def create_moran_quad_hex_map(gdf_agg, mode_col, value_cols, h3_resolution):
    gdf = gdf_agg.copy(deep = True)
    #gdf[mode_col] = gdf[mode_col].astype(str)
    gdf["geometry"] = gdf.centroid

    # Define hexagons
    def latlon_to_hexagon(row, resolution):
        return h3.geo_to_h3(row.geometry.y, row.geometry.x, resolution)

    gdf['hexagon'] = gdf.apply(latlon_to_hexagon, resolution=h3_resolution, axis=1)

    def _mode(s):
        m = s.mode()
        if isinstance(m, Iterable):
            m = m[0]
        return m
    
    hexagon_gdf = gdf.groupby('hexagon').agg(
        {mode_col : _mode, "task_id" : "nunique", "nearby_building_count": "mean"}
        | {v: "median" for v in value_cols}
    ).reset_index()
    hexagon_gdf[mode_col] = hexagon_gdf[mode_col].astype(int)

    def hexagon_to_geometry(hexagon):
        vertices = h3.h3_to_geo_boundary(hexagon, geo_json=True)
        return Polygon(vertices)

    hexagon_gdf['geometry'] = hexagon_gdf['hexagon'].apply(hexagon_to_geometry)

    hexagon_gdf = gpd.GeoDataFrame(hexagon_gdf, geometry='geometry').set_crs(4326)

    # Create the map
    m = folium.Map(tiles="Esri.WorldImagery", location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], zoom_start=8)

    hexagon_geojson = hexagon_gdf.to_json()

    tooltip = GeoJsonTooltip(
        fields=['hexagon', 'task_id', mode_col, 'nearby_building_count'] + [v for v in value_cols],
        aliases=['Hexagon ID', 'Hex Building Count', mode_col, "Avg Nearby Building Count"] + [f"Median {v} Value" for v in value_cols],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )

    # creating the custom ramp
    #lisa_cm = cm.StepColormap(colors = lisa_colormap, vmin = 0, vmax = len(lisa_colormap)-1)

    def style_function(feature):
        fillval = feature['properties'][mode_col]
        fillval = int(fillval)
        return {
            'fillColor': lisa_colormap[fillval],
            'color': 'black',
            'weight': 0.25,
            'lineOpacity': 0.2,
            'fillOpacity': 0.7,
        }
    
    # Add Choropleth layer
    # folium.Choropleth(
    #     geo_data=hexagon_geojson,
    #     name='choropleth',
    #     data=hexagon_gdf,
    #     columns=['hexagon', mode_col],
    #     key_on='feature.properties.hexagon',
    #     style_function=style_function,
    #     #fill_color="YlOrRd",
    #     fill_opacity=0.7,
    #     line_opacity=0.2,
    #     legend_name='dominant local Moran quadrant'
    # ).add_to(m)
    
    folium.GeoJson(
        hexagon_geojson,
        #style_function=lambda x: {"fillColor": "YlOrRd", "color": "black", "weight": 1, "fillOpacity":0},
        style_function=style_function,
        tooltip=tooltip
    ).add_to(m)

    m._repr_html_ = lambda: m._parent._repr_html_(
    include_link=False, width='75%', height='400px'
    )

    
    # Add the legend to the map
    m.get_root().add_child(Legend(dict(enumerate(lisa_colormap)), "Local Moran Quadrant"))
    
    return m

In [ ]:
create_moran_quad_map(df_agg_moran_w, "moran_quad_350m", value_cols=["remap_score", "ols_resid", "agreement"])

In [ ]:
df_agg_moran_w = df_agg_w.set_index("task_id").join(df_moran_local_w, how="inner").reset_index()
len(df_agg_w), len(df_moran_local_w), len(df_agg_moran_w)

In [ ]:
create_moran_quad_hex_map(df_agg_moran_w, mode_col="moran_quad_350m", value_cols=["remap_score", "agreement"], h3_resolution=11)

# Regression Models

## OLS

In [ ]:
from statsmodels.formula.api import ols

ind_vars = ["geom_segment_count", "nearby_building_count_log", "building_area_m2", "aspect_ratio", "coverage_ratio"]
#ind_vars = ["building_area_m2", "nearby_building_count", "year"]
#ind_vars = ["building_area_m2", "nearby_building_count", "year", "aspect_ratio"]
#ind_vars = ["nearby_building_count", "year", "aspect_ratio"]
#ind_vars = ["nearby_building_count", "year", "minx", "miny"]
#ind_vars = ["building_area_m2", "nearby_building_count", "year", "aspect_ratio", "minx", "miny"]
#ind_vars = ["building_area_m2", "nearby_building_count", "year", "minx", "miny"]
#ind_vars = ["building_area_m2", "nearby_building_count", "year", "aspect_ratio", "minx", "maxx", "miny", "maxy"]
df_ols = df_agg_w.copy()
df_ols = pd.concat([df_ols, df_ols.to_crs(df_ols.estimate_utm_crs()).geometry.bounds], axis=1)
#df_ols[ind_vars] = df_ols[ind_vars] / df_ols[ind_vars].max()
df_ols[ind_vars] = df_ols[ind_vars] - df_ols[ind_vars].mean()

#model = ols('Q("1_share") ~ building_area_m2 + nearby_building_count + year', data=df_agg_w).fit()
model = ols("remap_score ~ " + " + ".join(ind_vars), data=df_ols).fit()

In [ ]:
model.summary()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def check_multicollinearity(df, cols):
    df = df[cols]
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i)
                       for i in range(df.shape[1])]
    return vif_data

In [ ]:
check_multicollinearity(df_ols, ind_vars)

In [ ]:
df_agg_w["adjusted_remap_score"] = model.resid

In [ ]:
df_moran_local_w = calc_moran_local_for_dist(df_agg_w, "adjusted_remap_score", [500.0])

In [ ]:
df_agg_moran_w = df_agg_w.set_index("task_id").join(df_moran_local_w, how="inner").reset_index()

In [ ]:
df_agg_moran_w["adjusted_remap_score"].describe()

In [ ]:
df_agg_moran_w["1_share"].describe()

In [ ]:
df_agg_moran_w.dtypes

In [ ]:
create_moran_quad_hex_map(df_agg_moran_w, mode_col="moran_quad_500m", value_cols=ind_vars + ["adjusted_remap_score"], h3_resolution=11)

In [ ]:
#df_agg_moran_w = pd.concat([df_agg_moran_w, df_agg_moran_w.to_crs(df_ols.estimate_utm_crs()).geometry.bounds], axis=1)
create_moran_quad_map(df_agg_moran_w, "moran_quad_500m", value_cols=ind_vars + ["remap_score", "adjusted_remap_score"])

## Fixed Effect Regime

In [ ]:
import h3
import math
from pysal.lib import weights
from pysal.model import spreg
    

def model_ols_fe(gdf_agg_w, y_col, feature_cols, fe_h3_resolution):
    gdf = gdf_agg_w[feature_cols + ["geometry", y_col]].copy()
    
    gdf["geometry"] = gdf.to_crs(gdf.estimate_utm_crs()).centroid.to_crs(gdf_agg_w.crs)
    
    def latlon_to_hexagon(row, resolution):
        return h3.geo_to_h3(row.geometry.y, row.geometry.x, resolution)

    gdf["fe_hexbin"] = gdf.apply(latlon_to_hexagon, resolution=fe_h3_resolution, axis=1)

    # X = gdf[feature_cols + ["fe_hexbin"]]
    # dummies = pd.get_dummies(gdf, columns=["fe_hexbin"], prefix='_d', drop_first=False)
    # X = pd.concat([X, dummies], axis=1)
    
    # y = gdf[y_col]
    #dist = math.sqrt(h3.hex_area(fe_h3_resolution, unit="m^2") / math.pi)
    # dist = 100.0
    # w = weights.DistanceBand.from_dataframe(gdf, threshold=dist, binary=False)
    # w.transform = "R"
    
    # Fit the model
    # model = spreg.OLS(
    #     y, 
    #     X, 
    #     w=w, 
    #     name_y=y_col, 
    #     name_x=X.columns.tolist(), 
    #     name_w='fe_neighbors'
    # )

    # spreg spatial fixed effect implementation
    model = spreg.OLS_Regimes(
        # Dependent variable
        y=gdf[[y_col]].values,
        # Independent variables
        x=gdf[feature_cols].values,
        # Variable specifying neighborhood membership
        regimes=gdf["fe_hexbin"].tolist(),
        # TODO adding w when fe_hexbin is basically the same might be a mistake
        # w=w,
        # Allow the constant term to vary by group/regime
        constant_regi="many",
        # Variables to be allowed to vary (True) or kept
        # constant (False). Here we set all to False
        cols2regi=[False] * len(feature_cols),
        # Allow separate sigma coefficients to be estimated
        # by regime (False so a single sigma)
        regime_err_sep=False,
        # Dependent variable name
        name_y=y_col,
        # Independent variables names
        name_x=feature_cols,
    )
    
    # Print the summary
    #print(model.summary)

    return model, gdf

In [ ]:
m1, m1_dbg_gdf = model_ols_fe(df_agg_w, "incorrect_score", ["year", "building_area_m2", "nearby_building_count"], 8)

In [ ]:
print(m1.summary)

In [ ]:
import h3
import math
from pysal.lib import weights
from pysal.model import spreg
    

def model_ols_fe2(gdf_agg_w, y_col, feature_cols, fe_h3_resolution, w_dist_m):
    gdf = gdf_agg_w[feature_cols + ["geometry", y_col]].copy()
    
    gdf["geometry"] = gdf.to_crs(gdf.estimate_utm_crs()).centroid.to_crs(gdf_agg_w.crs)
    
    def latlon_to_hexagon(row, resolution):
        return h3.geo_to_h3(row.geometry.y, row.geometry.x, resolution)

    gdf["fe_hexbin"] = gdf.apply(latlon_to_hexagon, resolution=fe_h3_resolution, axis=1)

    w = weights.DistanceBand.from_dataframe(gdf.to_crs(gdf.estimate_utm_crs()), threshold=w_dist_m, binary=False)
    w.transform = "R"
    
    # Fit the model
    # model = spreg.OLS(
    #     y, 
    #     X, 
    #     w=w, 
    #     name_y=y_col, 
    #     name_x=X.columns.tolist(), 
    #     name_w='fe_neighbors'
    # )

    # spreg spatial fixed effect implementation
    model = spreg.OLS_Regimes(
        # Dependent variable
        y=gdf[[y_col]].values,
        # Independent variables
        x=gdf[feature_cols].values,
        # Variable specifying neighborhood membership
        regimes=gdf["fe_hexbin"].tolist(),
        # TODO adding w when fe_hexbin is basically the same might be a mistake
        w=w,
        # Allow the constant term to vary by group/regime
        constant_regi="many",
        # Variables to be allowed to vary (True) or kept
        # constant (False). Here we set all to False
        cols2regi=[False] * len(feature_cols),
        # Allow separate sigma coefficients to be estimated
        # by regime (False so a single sigma)
        regime_err_sep=False,
        # Dependent variable name
        name_y=y_col,
        # Independent variables names
        name_x=feature_cols,
    )
    
    # Print the summary
    #print(model.summary)

    return model, gdf

In [ ]:
m1_1, m1_1_dbg_gdf = model_ols_fe2(
    df_agg_w, 
    "incorrect_score", 
    ["year", "building_area_m2", "nearby_building_count"], 
    7,
    w_dist_m=50.0,
)

In [ ]:
print(m1_1.summary)

## Spatial lag of dependent variable
https://geographicdata.science/book/notebooks/11_regression.html#spatial-lag

In [ ]:
import h3
import math
from pysal.lib import weights
from pysal.model import spreg
    

def model_ols_depvar(gdf_agg_w, y_col, feature_cols):
    gdf = gdf_agg_w[feature_cols + ["geometry", y_col]]
    
    # y = gdf[y_col]
    #dist = math.sqrt(h3.hex_area(fe_h3_resolution, unit="m^2") / math.pi)
    # dist = 100.0
    w = weights.KNN.from_dataframe(gdf, k=20)
    # w.transform = "R"

    model = spreg.GM_Lag(
        # Dependent variable
        y=gdf[[y_col]].values,
        # Independent variables
        x=gdf[feature_cols].values,
        w=w,
        # Dependent variable name
        name_y=y_col,
        # Independent variables names
        name_x=feature_cols,
    )
    
    # Print the summary
    #print(model.summary)

    return model

In [ ]:
m2 = model_ols_depvar(df_agg_w, "remap_score", ["year", "building_area_m2", "nearby_building_count"])

In [ ]:
print(m2.summary)

In [ ]:
import h3
import math
from pysal.lib import weights
from pysal.model import spreg
    

def model_ols_depvar2(gdf_agg_w, y_col, feature_cols):
    gdf = gdf_agg_w[feature_cols + ["geometry", y_col]]
    gdf = gdf.to_crs(gdf.estimate_utm_crs())

    dist = 1000.0
    w = weights.DistanceBand.from_dataframe(gdf, threshold=dist, binary=True)
    w.transform = "R"

    model = spreg.GM_Lag(
        # Dependent variable
        y=gdf[[y_col]].values,
        # Independent variables
        x=gdf[feature_cols].values,
        w=w,
        # Dependent variable name
        name_y=y_col,
        # Independent variables names
        name_x=feature_cols,
    )
    
    # Print the summary
    #print(model.summary)

    return model

In [ ]:
m2_1 = model_ols_depvar2(df_agg_w, "remap_score", ["year", "building_area_m2", "nearby_building_count"])

In [ ]:
print(m2_1.summary)

## Model with endogenous features - KNN weights

In [ ]:
import h3
import math
from pysal.lib import weights
from pysal.model import spreg
    

def model_ols_endo1(gdf_agg_w, y_col, exog_cols, endo_col, instrument_cols):
    gdf = gdf_agg_w[exog_cols + instrument_cols + ["geometry", endo_col, y_col]]

    w = weights.KNN.from_dataframe(gdf, k=10)
    
    model = spreg.GM_Lag(
        y=gdf[[y_col]].values,
        x=gdf[exog_cols].values,
        yend=gdf[[endo_col]].values,
        q=gdf[instrument_cols].values,
        w=w,
        name_y=y_col,
        name_x=exog_cols,
        name_yend=[endo_col],
        name_q=instrument_cols,
    )
    
    # Print the summary
    #print(model.summary)

    return model

In [ ]:
m3 = model_ols_endo1(
    df_agg_w, 
    "incorrect_score", 
    ["year", "building_area_m2", "nearby_building_count"],
    "agreement",
    ["total_count_uw"],
)

In [ ]:
print(m3.summary)

## Model with endogenous features - DistanceBand weights

In [ ]:
import h3
import math
from pysal.lib import weights
from pysal.model import spreg
    

def model_ols_endo2(gdf_agg_w, y_col, exog_cols, endo_cols, instrument_cols):
    gdf = gdf_agg_w[exog_cols + instrument_cols + endo_cols + ["geometry", y_col]]
    gdf = gdf.to_crs(gdf.estimate_utm_crs())

    dist = 150.0
    w = weights.DistanceBand.from_dataframe(gdf, threshold=dist, binary=False)
    w.transform = "R"
    # w = weights.KNN.from_dataframe(gdf, k=20)
    
    model = spreg.GM_Lag(
        y=gdf[[y_col]].values,
        x=gdf[exog_cols].values,
        yend=gdf[endo_cols].values,
        q=gdf[instrument_cols].values,
        w=w,
        name_y=y_col,
        name_x=exog_cols,
        name_yend=endo_cols,
        name_q=instrument_cols,
    )
    
    # Print the summary
    #print(model.summary)

    return model

In [ ]:
m4 = model_ols_endo2(
    df_agg_w, 
    y_col="incorrect_score", 
    exog_cols=["year", "building_area_m2", "nearby_building_count"],
    endo_cols=["agreement"],
    instrument_cols=["total_count_uw"],
)

In [ ]:
print(m4.summary)